# 텍스트 분류: CountVectorizer vs. TfidfVectorizer

이 노트북은 텍스트 데이터를 머신러닝 모델이 이해할 수 있는 숫자 형태로 변환하는 두 가지 주요 방법인 **`CountVectorizer`**와 **`TfidfVectorizer`**를 비교하고, 이를 로지스틱 회귀 모델에 적용하여 텍스트 분류를 수행하는 과정을 다룹니다.

**사용 데이터셋**: IMDb 영화 리뷰 데이터셋 (긍정/부정 분류)

### 1. 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split # 데이터 분할을 위해 추가

### 2. 데이터 로드 및 탐색

`scikit-learn`의 `load_files` 함수를 사용하여 IMDb 영화 리뷰 데이터를 불러옵니다. 이 데이터셋은 훈련(train)과 테스트(test) 폴더로 나뉘어 있으며, 각 폴더 안에는 긍정(pos)과 부정(neg) 리뷰가 포함되어 있습니다.

In [ ]:
try:
    reviews_train = load_files("./data/aclImdb/train")
    reviews_test = load_files("./data/aclImdb/test")
    print("데이터 로드 성공!")
except FileNotFoundError:
    print("오류: './data/aclImdb/train' 또는 './data/aclImdb/test' 폴더를 찾을 수 없습니다.")
    print("노트북이 있는 폴더(02_practice/0716) 아래에 'data/aclImdb/' 폴더를 만들고 train/test 데이터를 넣어주세요.")
    reviews_train, reviews_test = None, None

if reviews_train:
    text_train_raw, y_train = reviews_train.data, reviews_train.target
    text_test_raw, y_test = reviews_test.data, reviews_test.target

    print(f"훈련 데이터 샘플 수: {len(text_train_raw)}")
    print(f"테스트 데이터 샘플 수: {len(text_test_raw)}")
    print("
첫 번째 훈련 리뷰 원본:")
    print(text_train_raw[0])

### 3. 텍스트 데이터 전처리

원본 리뷰 데이터에는 `<br />`와 같은 HTML 태그가 포함되어 있습니다. 이러한 불필요한 문자열은 분석에 방해가 되므로 제거합니다.

In [ ]:
if reviews_train:
    text_train_cleaned = [doc.replace(b"<br />", b" ") for doc in text_train_raw]
    text_test_cleaned = [doc.replace(b"<br />", b" ") for doc in text_test_raw]

    print("정제 후 첫 번째 훈련 리뷰:")
    print(text_train_cleaned[0])

### 4. 텍스트 벡터화 (CountVectorizer)

`CountVectorizer`는 문서에서 단어의 출현 횟수를 세어 벡터로 변환합니다. 각 단어는 고유한 특성(feature)이 되며, 벡터의 값은 해당 단어가 문서에 나타난 횟수입니다.

In [ ]:
if reviews_train:
    # CountVectorizer 학습 및 변환
    vect_count = CountVectorizer().fit(text_train_cleaned)
    X_train_count = vect_count.transform(text_train_cleaned)
    X_test_count = vect_count.transform(text_test_cleaned)

    print(f"CountVectorizer 특성(단어) 개수: {len(vect_count.get_feature_names_out())}")
    print(f"훈련 데이터 벡터 형태 (Count): {X_train_count.shape}")

    # 로지스틱 회귀 모델 학습 및 평가
    model_count = LogisticRegression(max_iter=5000, random_state=0)
    model_count.fit(X_train_count, y_train)

    train_score_count = model_count.score(X_train_count, y_train)
    test_score_count = model_count.score(X_test_count, y_test)

    print(f"CountVectorizer 기반 모델 훈련 정확도: {train_score_count:.4f}")
    print(f"CountVectorizer 기반 모델 테스트 정확도: {test_score_count:.4f}")

### 5. 텍스트 벡터화 (TfidfVectorizer)

`TfidfVectorizer`는 단순히 단어의 빈도수만 세는 것이 아니라, TF-IDF(Term Frequency-Inverse Document Frequency) 가중치를 사용하여 단어의 중요도를 반영합니다.

- **TF (Term Frequency)**: 특정 문서 내에서 단어가 나타나는 빈도
- **IDF (Inverse Document Frequency)**: 단어가 전체 문서 집합에서 얼마나 희귀하게 나타나는지 (희귀할수록 높은 값)

TF-IDF는 특정 문서에서는 자주 나타나지만 전체 문서에서는 흔하지 않은 단어에 높은 가중치를 부여하여, 문서의 특징을 더 잘 나타내도록 합니다.

In [ ]:
if reviews_train:
    # TfidfVectorizer 학습 및 변환
    vect_tfidf = TfidfVectorizer().fit(text_train_cleaned)
    X_train_tfidf = vect_tfidf.transform(text_train_cleaned)
    X_test_tfidf = vect_tfidf.transform(text_test_cleaned)

    print(f"TfidfVectorizer 특성(단어) 개수: {len(vect_tfidf.get_feature_names_out())}")
    print(f"훈련 데이터 벡터 형태 (TF-IDF): {X_train_tfidf.shape}")

    # 로지스틱 회귀 모델 학습 및 평가
    model_tfidf = LogisticRegression(max_iter=5000, random_state=0)
    model_tfidf.fit(X_train_tfidf, y_train)

    train_score_tfidf = model_tfidf.score(X_train_tfidf, y_train)
    test_score_tfidf = model_tfidf.score(X_test_tfidf, y_test)

    print(f"TfidfVectorizer 기반 모델 훈련 정확도: {train_score_tfidf:.4f}")
    print(f"TfidfVectorizer 기반 모델 테스트 정확도: {test_score_tfidf:.4f}")

### 6. 결과 비교 및 결론

두 가지 벡터화 방법의 테스트 정확도를 비교해 봅시다.

In [ ]:
if reviews_train:
    print("
--- 모델 테스트 정확도 비교 ---")
    print(f"CountVectorizer 기반 모델: {test_score_count:.4f}")
    print(f"TfidfVectorizer 기반 모델: {test_score_tfidf:.4f}")

    print("
--- 결론 ---")
    print("일반적으로 TfidfVectorizer가 CountVectorizer보다 더 좋은 성능을 보이는 경우가 많습니다.")
    print("이는 TfidfVectorizer가 단어의 단순 빈도뿐만 아니라 문서 내에서의 중요도와 전체 문서 집합에서의 희귀도를 함께 고려하기 때문입니다.")
    print("하지만 어떤 벡터화 방법이 더 적합한지는 데이터셋의 특성과 해결하려는 문제에 따라 달라질 수 있습니다.")